CNN Model code which require google drive access for dataset

In [ ]:
import zipfile
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '13wgU2AtpPQvyzJKQNWX5NUIhnq391BOR'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.zip')

with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

def load_images(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        if os.path.isfile(img_path):
            img = load_img(img_path, target_size=(height, width))
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

height, width = 255, 255
channels = 3

bad_images, bad_labels = load_images('/content/data/output/bad', label=0)
good_images, good_labels = load_images('/content/data/output/good', label=1)

all_images = np.concatenate([bad_images, good_images], axis=0)
all_labels = np.concatenate([bad_labels, good_labels], axis=0)

train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

train_images, test_images = train_images / 255.0, test_images / 255.0

model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(height, width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

adam_optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

train_labels_one_hot = to_categorical(train_labels)
test_labels_one_hot = to_categorical(test_labels)

model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32, validation_split=0.2)

test_loss, test_acc = model.evaluate(test_images, test_labels_one_hot)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
29/29 [==============================] - 6s 116ms/step - loss: 0.6656 - accuracy: 0.6243 - val_loss: 0.5918 - val_accuracy: 0.5844
Epoch 2/10
29/29 [==============================] - 3s 90ms/step - loss: 0.5077 - accuracy: 0.7752 - val_loss: 0.5855 - val_accuracy: 0.7489
Epoch 3/10
29/29 [==============================] - 3s 93ms/step - loss: 0.3803 - accuracy: 0.8187 - val_loss: 0.3427 - val_accuracy: 0.8355
Epoch 4/10
29/29 [==============================] - 3s 88ms/step - loss: 0.3979 - accuracy: 0.8306 - val_loss: 0.3504 - val_accuracy: 0.7965
Epoch 5/10
29/29 [==============================] - 3s 90ms/step - loss: 0.2902 - accuracy: 0.8643 - val_loss: 0.2369 - val_accuracy: 0.9048
Epoch 6/10
29/29 [==============================] - 3s 95ms/step - loss: 0.2488 - accuracy: 0.8914 - val_loss: 0.2085 - val_accuracy: 0.9048
Epoch 7/10
29/29 [==============================] - 3s 92ms/step - loss: 0.2161 - accuracy: 0.9077 - val_loss: 0.1261 - val_accuracy: 0.9264
Epoch 8/10
2

Code for manual testing

In [ ]:
import zipfile
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# file_id = '17ReCiLc6NsrtOUhG4qGkbWjDmid9xZDM'
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('Good.zip')

# file_id = '1XBdO6dM8IyNQ6-HUGGwRCPQajxx30Xtz'
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('Bad.zip')

# with zipfile.ZipFile('Good.zip', 'r') as zip_ref:
#     zip_ref.extractall('Good')

# test_folder_path = '/content/Good/Good/'

with zipfile.ZipFile('Bad.zip', 'r') as zip_ref:
    zip_ref.extractall('Bad')

test_folder_path = '/content/Bad/Bad/'

for image_name in os.listdir(test_folder_path):
    image_path = os.path.join(test_folder_path, image_name)

    img = load_img(image_path, target_size=(height, width))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        print(prediction[0][0])
        print(f"Image {image_name}: Predicted as Good")
    else:
        print(prediction[0][0])
        print(f"Image {image_name}: Predicted as Bad")


1/1 [==============================] - 0s 31ms/step
0.9946956
Image BAD_21.png: Predicted as Good
1/1 [==============================] - 0s 36ms/step
0.025447687
Image BAD_145.png: Predicted as Bad
1/1 [==============================] - 0s 27ms/step
6.209581e-08
Image BAD_178.png: Predicted as Bad
1/1 [==============================] - 0s 25ms/step
0.0023283006
Image BAD_176.png: Predicted as Bad
1/1 [==============================] - 0s 17ms/step
0.31944823
Image 4k_3 4.png: Predicted as Bad
1/1 [==============================] - 0s 17ms/step
0.0005550079
Image 5k_2 1.png: Predicted as Bad
1/1 [==============================] - 0s 20ms/step
0.14401662
Image 4k_3 5.png: Predicted as Bad
1/1 [==============================] - 0s 23ms/step
0.3946886
Image BAD_134.png: Predicted as Bad
1/1 [==============================] - 0s 19ms/step
2.3653334e-13
Image BAD_179.png: Predicted as Bad
1/1 [==============================] - 0s 22ms/step
0.7574769
Image BAD_140.png: Predicted as Good
1/1 [

In [ ]:
import shutil
import os

folder_path = '/content/test/'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' and its contents have been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")

Folder '/content/test/' and its contents have been deleted.


STFT generation code

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib import colors
import os

def calcSTFT_norm(df, column_name, samplingFreq=10000, window='hann', nperseg=100000, figsize=(10,6), cmap='viridis', ylim_max=None, save_path=None):
    inputSignal = df[column_name].values

    if len(inputSignal) < nperseg:
        print(f"Warning: nperseg = {nperseg} is greater than input length = {len(inputSignal)}. Skipping STFT calculation.")
        return

    num_chunks = len(inputSignal) // nperseg
    remainder = len(inputSignal) % nperseg

    for i in range(num_chunks):
        start_idx = i * nperseg
        end_idx = (i + 1) * nperseg
        chunk_signal = inputSignal[start_idx:end_idx]

        f, t, Zxx = signal.stft(chunk_signal, samplingFreq, window=window, nperseg=nperseg)

        fig = plt.figure(figsize=figsize)
        spec = plt.pcolormesh(t, f, np.abs(Zxx),
                              norm=colors.PowerNorm(gamma=1./6.),
                              cmap=plt.get_cmap(cmap))
        cbar = plt.colorbar(spec)

        ax = fig.axes[0]
        ax.set_xlim(0, t[-1])

        plt.title(f'STFT Spectrogram for {column_name} - Chunk {i+1}')
        ax.grid(True)
        ax.set_title('STFT Magnitude')
        if ylim_max:
            ax.set_ylim(0, ylim_max)
        ax.set_ylabel('Frequency [Hz]')
        ax.set_xlabel('Time [sec]')

        if save_path:
            plt.savefig(f'{save_path[:-4]} {i+1}.png')
        else:
            plt.show()

    if remainder > 0:
        remaining_signal = inputSignal[-remainder:]
        f, t, Zxx = signal.stft(remaining_signal, samplingFreq, window=window, nperseg=nperseg)

        fig = plt.figure(figsize=figsize)
        spec = plt.pcolormesh(t, f, np.abs(Zxx),
                              norm=colors.PowerNorm(gamma=1./6.),
                              cmap=plt.get_cmap(cmap))
        cbar = plt.colorbar(spec)

        ax = fig.axes[0]
        ax.set_xlim(0, t[-1])

        plt.title(f'STFT Spectrogram for {column_name} - Remaining Data')
        ax.grid(True)
        ax.set_title('STFT Magnitude')
        if ylim_max:
            ax.set_ylim(0, ylim_max)
        ax.set_ylabel('Frequency [Hz]')
        ax.set_xlabel('Time [sec]')

        if save_path:
            plt.savefig(f'{save_path[:-4]}_remaining.png')
        else:
            plt.show()

df = pd.read_csv('/test.csv')


column_names = ['cDAQ9189-20796A8Mod1_ai1']

output_directory = r'P:\pr\Internship\test_op'

for i, column_name in enumerate(column_names, start=1):
    base_save_path = os.path.join(output_directory, f'test_{i}')

    save_path = f'{base_save_path}.png'
    calcSTFT_norm(df, column_name, save_path=save_path)


Data Preprocessing code - Resize

In [ ]:
from PIL import Image
import os

def crop_image(input_path, output_path, left, top, width, height):
    img = Image.open(input_path)

    right = left + width
    bottom = top + height

    cropped_img = img.crop((left, top, right, bottom))

    cropped_img.save(output_path)

def batch_crop_images(input_folder, output_folder, crop_info):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            crop_image(input_path, output_path, *crop_info)

input_folder = "P:\\pr\\Internship\\output\\good"
output_folder = "P:\\pr\\Internship\\output\\good"
crop_info = (123, 71, 685, 466)  # (left, top, width, height)
# 9:5  image dimensions crop_info = (112, 59, 615, 389)  # (left, top, width, height)
# 10:6 image dimensions crop_info = (123, 71, 685, 466)  # (left, top, width, height)

batch_crop_images(input_folder, output_folder,crop_info)

Data Preprocessing -